In [1]:
def get_valid_int(prompt, min_value=None, max_value=None):
    while True:
        try:
            user_input = int(input(prompt))
            if (min_value is None or user_input >= min_value) and (
                max_value is None or user_input <= max_value
            ):
                return user_input
            else:
                if min_value is not None and max_value is not None:
                    print(
                        f"Please enter an integer between {min_value} and {max_value}."
                    )
                elif min_value is not None:
                    print(
                        f"Please enter an integer greater than or equal to {min_value}."
                    )
                elif max_value is not None:
                    print(f"Please enter an integer less than or equal to {max_value}.")
        except ValueError:
            print("Invalid input. Please enter a valid integer.")


def get_valid_float(prompt, min_value=None, max_value=None):
    while True:
        try:
            user_input = float(input(prompt))
            if (min_value is None or user_input >= min_value) and (
                max_value is None or user_input <= max_value
            ):
                return user_input
            else:
                if min_value is not None and max_value is not None:
                    print(f"Please enter a number between {min_value} and {max_value}.")
                elif min_value is not None:
                    print(
                        f"Please enter a number greater than or equal to {min_value}."
                    )
                elif max_value is not None:
                    print(f"Please enter a number less than or equal to {max_value}.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")


waiver_ask_destroy = (
    "Percentage of involved people who'd likely want to waive consideration of this value "
    "destruction for them if they knew of a bigger corresponding value destruction? "
)
waiver_ask_build = (
    "Percentage of involved people who'd likely want to waive consideration of this value "
    "build for them if they knew of a bigger corresponding value destruction? "
)

In [2]:
def calc_destroy(top_destroy, num_pers, life_val, waiver, prop_val):
    top_destroy_wt = 0.0
    if top_destroy == 1:
        # 1 - Someone dying, or increasing its risk
        print("For someone dying:")
        if num_pers == 0:
            n_die = get_valid_int(
                "How many effectively equivalent people would die? ", min_value=0
            )
            waiver = get_valid_float(waiver_ask_destroy, min_value=0, max_value=100)
            defaults = input(
                "Do you want to use default values for this person's life value (Y or N)? "
            )
            if defaults == "Y":
                destroy = 1.54586
            else:
                destroy = life_value_fn()
        else:
            n_die = num_pers
            destroy = life_val
        top_destroy_wt = n_die * destroy * (1.0 - waiver / 100.0)
        print(f"*****Value destruction weight is: {top_destroy_wt}")
    elif top_destroy == 2:
        # 2 - Non-freely chosen physical pain, or increasing its risk
        print("For unwanted physical pain:")
        if num_pers == 0:
            n_pain = get_valid_int(
                "How many effectively equivalent people would experience pain? ",
                min_value=0,
            )
            waiver = get_valid_float(waiver_ask_destroy, min_value=0, max_value=100)
        else:
            n_pain = num_pers
        destroy = unwanted_pain_fn()
        top_destroy_wt = n_pain * destroy * (1.0 - waiver / 100.0)
        print(f"*****Value destruction weight is: {top_destroy_wt}")
    elif top_destroy == 3:
        # 3 - Long-term health issue, or increasing its risk
        print("For long-term health issue:")
        if num_pers == 0:
            n_health = get_valid_int(
                "How many effectively equivalent people would have long-term health issue? ",
                min_value=0,
            )
            waiver = get_valid_float(waiver_ask_destroy, min_value=0, max_value=100)
        else:
            n_health = num_pers
        destroy = longterm_health_fn()
        top_destroy_wt = n_health * destroy * (1.0 - waiver / 100.0)
        print(f"*****Value destruction weight is: {top_destroy_wt}")
    elif top_destroy == 4:
        # 4 - Damaging or destroying property
        print("For damage or destruction of property:")
        if num_pers == 0:
            n_prop_destroy = get_valid_int(
                "How many effectively equivalent people would have their property damaged? ",
                min_value=0,
            )
            waiver = get_valid_float(waiver_ask_destroy, min_value=0, max_value=100)
            destroy = steal_and_property_destroy_fn()
        else:
            n_prop_destroy = num_pers
            destroy = prop_val
        top_destroy_wt = n_prop_destroy * destroy * (1.0 - waiver / 100.0)
        print(f"*****Value destruction weight is: {top_destroy_wt}")
    elif top_destroy == 5:
        # 5 - Going against one’s conscience
        print("For going against one's conscience:")
        if num_pers == 0:
            n_consc = get_valid_int(
                "How many effectively equivalent people would go against their conscience? ",
                min_value=0,
            )
            waiver = get_valid_float(waiver_ask_destroy, min_value=0, max_value=100)
        else:
            n_consc = num_pers
        destroy = conscience_fn()
        top_destroy_wt = n_consc * destroy * (1.0 - waiver / 100.0)
        print(f"*****Value destruction weight is: {top_destroy_wt}")
    return top_destroy_wt

In [3]:
def life_value_fn():
    # This is a very rough and incomplete approximation to life value for these proof-of-concept calculations
    age = get_valid_float("Age (in years) of person? ", min_value=0, max_value=125)
    qol = get_valid_float(
        "Average quality of remaining life (-1 to 1)? ", min_value=-1, max_value=1
    )
    income = get_valid_float("Yearly income (in $) of person? ")
    n_interact = get_valid_float(
        "Average number of people per year they would interact with if they live? ",
        min_value=0,
    )
    value_add = get_valid_float(
        "Average value add per person they would interact with (can be negative; enter 0.0001 if unsure)? "
    )
    n_loss = get_valid_float(
        "How many relationships do they have with people who would feel the emotional loss? ",
        min_value=0,
    )
    emot_depth = get_valid_float(
        "Average depth of relationships, for the emotional loss felt if they die (0 to 1)? ",
        min_value=0,
        max_value=1,
    )
    # From curve fit to data from https://www.ssa.gov/oact/STATS/table4c6.html, averaging male and female
    life_expect = 3.956 * 10**-5 * age**3 - 0.0023 * age**2 + 0.0767 * age + 76.98
    print(f"Life expectancy calculated to be {life_expect} years.")
    if age < 66:
        life_val = (
            (65 - age) * income / 4050000
            + (life_expect - age) * (qol / 54 + n_interact * value_add)
            + 0.001 * emot_depth * n_loss
        )
    elif age < 71:
        life_val = (life_expect - age) * (
            qol / 54 + n_interact * value_add
        ) + 0.001 * emot_depth * n_loss
    else:
        life_val = (life_expect - age) * (
            qol / 54 + n_interact * value_add
        ) + 0.0001 * emot_depth * n_loss
    print(f"*****Life value is: {life_val}")
    return life_val


def unwanted_pain_fn():
    print(
        "Pain levels: 0 = No pain, 1 = Hardly notice, 2 = Notice, doesn’t interfere with activities, "
        "3 = Sometimes distracts, 4 = Distracts, can do usual activities, 5 = Interrupts some activities, "
        "6 = Hard to ignore, avoid usual activities, 7 = Focus of attention, prevents doing daily activities, "
        "8 = Awful, hard to do anything, 9 = Can’t bear the pain, unable to do anything, "
        "10 = As bad as could be, nothing else matters"
    )
    pain_level = get_valid_int(
        "Physical pain level (0 to 10)? ", min_value=0, max_value=10
    )
    if pain_level != 0:
        duration = get_valid_float(
            "Duration of physical pain (in minutes)? ", min_value=0
        )
        unbuilt_value = get_valid_float(
            "Net value could've been built (in $) if not for unwanted pain? ",
            min_value=0,
        )
        if pain_level > 7:
            destroy_wt_pain = (1.903 * 10**-6) * (
                duration * 100 ** (pain_level - 7)
            ) + unbuilt_value / 4050000.0
        else:
            destroy_wt_pain = (1.903 * 10**-13) * (
                duration * 10**pain_level
            ) + unbuilt_value / 4050000.0
    else:
        destroy_wt_pain = 0.0
    epain_level = get_valid_int(
        "Emotional pain level (0 to 10)? ", min_value=0, max_value=10
    )
    if epain_level != 0:
        eduration = get_valid_float(
            "Duration of emotional pain (in minutes)? ", min_value=0
        )
        unbuilt_evalue = get_valid_float(
            "Net value could've been built (in $) if not for unwanted pain? ",
            min_value=0,
        )
        if epain_level > 7:
            destroy_wt_epain = (1.903 * 10**-6) * (
                eduration * 100 ** (epain_level - 7)
            ) + unbuilt_evalue / 4050000.0
        else:
            destroy_wt_epain = (1.903 * 10**-13) * (
                eduration * 10**epain_level
            ) + unbuilt_evalue / 4050000.0
    else:
        destroy_wt_epain = 0.0
    destroy_wt = destroy_wt_pain + destroy_wt_epain
    return destroy_wt


def longterm_health_fn():
    years = get_valid_float(
        "Expected duration of health issue (in years)? ", min_value=0, max_value=125
    )
    if years != 0:
        qol_change = get_valid_float(
            "Average quality of life decrease during health issue (0 to 2)? ",
            min_value=0,
            max_value=2,
        )
        income = get_valid_float(
            "Yearly income (in $) of person who would have health issue? ", min_value=0
        )
        age = get_valid_int(
            "Age (in years) of person who would have health issue? ",
            min_value=0,
            max_value=125,
        )
        potential_lost = get_valid_float(
            "Percent of full potential lost due to health issue? ",
            min_value=0,
            max_value=100,
        )
        work_years = 65 - age
        if work_years <= 0:
            destroy_wt = years * qol_change / 54.0
        elif work_years >= years:
            destroy_wt = (
                years * income * (1.0 - potential_lost / 100.0) / 4050000.0
                + years * qol_change / 54.0
            )
        else:
            destroy_wt = (
                work_years * income * (1.0 - potential_lost / 100.0) / 4050000.0
                + years * qol_change / 54.0
            )
    else:
        destroy_wt = 0.0
    return destroy_wt


def steal_and_prop_destroy_fn():
    cost = get_valid_float("Replacement cost in dollars? ", min_value=0)
    if cost != 0.0:
        survive_change = get_valid_float(
            "Percent by which the property loss decreases survival likelihood over the next year? ",
            min_value=0,
            max_value=100,
        )
        weight_death = get_valid_float(
            "Value weight of person(s) dying in the next year (enter 1.54586 if unsure)? "
        )
        eloss = get_valid_float(
            "Weight of emotional loss if property had special meaning (enter 0.0001 if unsure)? ",
            min_value=0,
        )
        lessbuild = get_valid_float(
            "Likely decreased future value building (in $) due to property loss, over next year? ",
            min_value=0,
        )
        destroy_wt = (
            cost / 4050000.0
            + survive_change / 100 * weight_death
            + eloss
            + lessbuild / 4050000.0
        )
    else:
        destroy_wt = 0.0
    return destroy_wt


def conscience_fn():
    print(
        "Ethical breach levels: 1 - telling white lies, going against small values/ethics, "
        "2 - not helping others in significant need (who you don't see) when you could, "
        "3 - lying, stealing, cheating, not keeping your word, 4 - being significantly violent, "
        "not helping someone in significant need (right in front of you) when you could, "
        "5 - going against a large value/important ethic, such as not to murder, "
        "6 - extreme torturing"
    )
    breach_level = get_valid_int(
        "Level of ethical breach (1 to 6)? ", min_value=1, max_value=6
    )
    num_prev_breach = get_valid_int(
        "Previous number of breaches at this level? ", min_value=0
    )
    if breach_level == 1:
        destroy_wt = 0.0001 / (num_prev_breach + 1)
    elif breach_level == 2:
        destroy_wt = 0.005 / (num_prev_breach + 1)
    elif breach_level == 3:
        destroy_wt = 0.05 / (num_prev_breach + 1)
    elif breach_level == 4:
        destroy_wt = 0.5 / (num_prev_breach + 1)
    elif breach_level == 5:
        destroy_wt = 2.0 / (num_prev_breach + 1)
    else:
        destroy_wt = 5.0 / (num_prev_breach + 1)
    return destroy_wt

In [4]:
def life_rights_fn(villain, villain_prop, percent_culp, harms, num_pers):
    print("For violation of right to life:")
    viol_type = input(
        "Would this be an actual violation (A) of right to life or just a threatened (T) one (A or T)? "
    )
    if viol_type != "A":
        tlikelihood = get_valid_float(
            "Perceived percent likelihood that the person making the threat would go through with it? ",
            min_value=0,
            max_value=100,
        )
    else:  # Default is to assume it is an actual violation of right to life
        tlikelihood = 100.0
    worth_less = input(
        "Has the person made clear that they want their life to be worth less than those of others involved (Y or N)? "
    )
    if worth_less == "Y":
        print(
            "If person purposely put themselves in harm's way to commit suicide, they effectively want their life valued at zero."
        )
        want_valued = get_valid_float(
            "What fraction do they want their right to life valued at with respect to others? (0=not at all, 1= equally) ",
            min_value=0,
            max_value=1,
        )
    else:
        want_valued = 1.0
    if num_pers == 0:
        num_viol = get_valid_int(
            "How many people's (including this one's) right to life would be violated at the same level of culpability? ",
            min_value=0,
        )
    else:
        num_viol = num_pers
    if villain == 1:
        # If they put someone's life or body integrity in harm's way, they're considered to have 1/10,000th of rights of someone 100% in harm's way
        harms = 100.0
        percent_culp = max(percent_culp, 99.99)
    elif villain_prop == 1:
        # If only put someone's property in harm's way, they're considered to have rights of someone 100% in harm's way
        harms = 100.0
        percent_culp = max(percent_culp, 0.0)
    # People 100% harm's way have right to life weight of 100, those 0% in harm's way have weight of 1,000,000
    top_destroy_wt = (
        num_viol
        * want_valued
        * (tlikelihood / 100)
        * 1000000.0
        * (1 - (harms / 100) * (percent_culp / 100))
        / (1 + 9999 * (harms / 100))
    )
    print(f"*****Value destruction weight is: {top_destroy_wt}")
    return top_destroy_wt


def body_rights_fn(villain, villain_prop, percent_culp, harms, num_pers):
    print(
        "For violation of right to body integrity, including pain, health issues, etc.:"
    )
    viol_type = input(
        "Would this be an actual violation (A) of right to body integrity or just a threatened (T) one (A or T)? "
    )
    if viol_type != "A":
        tlikelihood = get_valid_float(
            "Perceived percent likelihood that person would make good on the threat? ",
            min_value=0,
            max_value=100,
        )
    else:
        tlikelihood = 100.0
    if num_pers == 0:
        num_viol = get_valid_int(
            "How many people's (including this one's) right to body integrity would be violated to the same level, including culpability? ",
            min_value=0,
        )
    else:
        num_viol = num_pers
    if num_viol != 0:
        print("For unwanted physical pain:")
        pain = unwanted_pain_fn()
        print("For long-term health issues:")
        health = longterm_health_fn()
        print("For threat of physical violence or emotional pain:")
        tpain = threat_pain_fn()
        print(
            "For threat of physical violence to the point of a long-term health issue:"
        )
        thealth = threat_longterm_health_fn()
        print("For threat of physical violence to the point of death:")
        tkill = threat_kill_fn()
        print("For physical restraint:")
        restraint = restraint_fn()
        worth_less = input(
            "Has the person made clear that they want their body integrity to be worth less than those of others involved (Y or N)? "
        )
        if worth_less == "Y":
            want_valued = get_valid_float(
                "How much do they want their right to body integrity valued with respect to others? (0=not at all, 1=equally) ",
                min_value=0,
                max_value=1,
            )
        else:
            want_valued = 1.0
        if villain == 1:
            # If they put someone's life or body integrity in harm's way, they're considered to have 1/10,000th of rights of someone 100% in harm's way
            harms = 100.0
            percent_culp = max(percent_culp, 99.99)
        elif villain_prop == 1:
            # If they only put someone's property in harm's way, they're considered to have rights of someone 100% in harm's way
            harms = 100.0
            percent_culp = max(percent_culp, 0.0)
        top_destroy_wt = (
            num_viol
            * want_valued
            * (tlikelihood / 100)
            * (
                1000000.0
                * (1 - (harms / 100) * (percent_culp / 100))
                * (pain + health + tpain + thealth + tkill + restraint)
            )
            / (1 + 9999 * (harms / 100))
        )
    else:
        top_destroy_wt = 0.0
    print(f"*****Value destruction weight is: {top_destroy_wt}")
    return top_destroy_wt, pain, health, tpain, thealth, tkill, restraint


def prop_rights_fn(villain, villain_prop, harm_prop, prop_val, num_pers):
    print("For violation of right to property:")
    viol_type = input(
        "Would this be an actual violation (A) of right to property or just a threatened (T) one (A or T)? "
    )
    if viol_type != "A":
        tlikelihood = get_valid_float(
            "Perceived percent likelihood that person would make good on the threat? ",
            min_value=0,
            max_value=100,
        )
    else:
        tlikelihood = 100.0
    if num_pers == 0:
        num_viol = get_valid_int(
            "How many people's (including this one's) right to property would be violated at the same level of culpability? ",
            min_value=0,
        )
        print("For property destroyed and/or stolen:")
        property_value = steal_and_prop_destroy_fn()
    else:
        num_viol = num_pers
        property_value = prop_val
    if harm_prop > 0:
        culp_prop = get_valid_float(
            "What's the person's culpability for their property-risking situation, in percent? ",
            min_value=0,
            max_value=100,
        )
    else:
        culp_prop = 0.0
    if villain == 1:
        # If they put someone's life or body integrity in harm's way, their rights drop to 1/10,000th the value for property in harm's way
        harm_prop = 100.0
        culp_prop = max(culp_prop, 99.99)
    elif villain_prop == 1:
        # If they only put someone's property in harm's way, their rights drop to 1/5th the value for property in harm's way
        harm_prop = 100.0
        culp_prop = max(culp_prop, 80.0)
    top_destroy_wt = (
        num_viol
        * (1 - harm_prop / 100 * culp_prop / 100)
        * (tlikelihood / 100)
        * (10000.0 * property_value)
        / (1 + 99 * (harm_prop / 100))
    )
    fix_intent = input(
        "Is there intent of those responsible to repair this property value destruction later (Y/N)? "
    )
    if fix_intent == "Y":
        likelihood_fix = get_valid_float(
            "What's the percent likelihood of this value destruction repair occurring? ",
            min_value=0,
            max_value=100,
        )
        percent_fix = get_valid_float(
            "What percentage of the value destruction is likely to be repaired (up to max of 99)? ",
            min_value=0,
            max_value=99,
        )
    else:
        likelihood_fix = 0.0
        percent_fix = 0.0
    top_destroy_wt_fixed = top_destroy_wt * (
        1 - likelihood_fix / 100.0 * percent_fix / 100.0
    )
    print(f"*****Value destruction weight (before repairing it) is: {top_destroy_wt}")
    return top_destroy_wt, property_value, top_destroy_wt_fixed

In [5]:
def threat_pain_fn():
    print(
        "Pain levels: 0 = No pain, 1 = Hardly notice, 2 = Notice, doesn’t interfere with activities, "
        "3 = Sometimes distracts, 4 = Distracts, can do usual activities, 5 = Interrupts some activities, "
        "6 = Hard to ignore, avoid usual activities, 7 = Focus of attention, prevents doing daily activities, "
        "8 = Awful, hard to do anything, 9 = Can’t bear the pain, unable to do anything, "
        "10 = As bad as could be, nothing else matters"
    )
    tpain_level = get_valid_int(
        "Threatened physical pain level (0 to 10)? ", min_value=0, max_value=10
    )
    if tpain_level != 0:
        tduration = get_valid_float(
            "Threatened duration of physical pain (in minutes)? ", min_value=0
        )
        tlikelihood = get_valid_float(
            "Perceived percent likelihood that person would make good on the threat? ",
            min_value=0,
            max_value=100,
        )
        unbuilt_value = get_valid_float(
            "Net value could build (in $) without unwanted pain? ", min_value=0
        )
        if tpain_level > 7:
            destroy_wt_pain = (
                0.5
                * (
                    (1.903 * 10**-6) * (tduration * 100 ** (tpain_level - 7))
                    + unbuilt_value / 4050000.0
                )
                * tlikelihood
                / 100.0
            )
        else:
            destroy_wt_pain = (
                0.5
                * (
                    (1.903 * 10**-13) * (tduration * 10**tpain_level)
                    + unbuilt_value / 4050000.0
                )
                * tlikelihood
                / 100.0
            )
    else:
        destroy_wt_pain = 0.0
    tepain_level = get_valid_int(
        "Threatened emotional pain level (0 to 10)? ", min_value=0, max_value=10
    )
    if tepain_level != 0:
        teduration = get_valid_float(
            "Threatened duration of emotional pain (in minutes)? ", min_value=0
        )
        telikelihood = get_valid_float(
            "Perceived percent likelihood that person would make good on the threat? ",
            min_value=0,
            max_value=100,
        )
        unbuilt_evalue = get_valid_float(
            "Net value could build (in $) without unwanted pain? ", min_value=0
        )
        if tepain_level > 7:
            destroy_wt_epain = (
                0.5
                * (
                    (1.903 * 10**-6) * (teduration * 100 ** (tepain_level - 7))
                    + unbuilt_evalue / 4050000.0
                )
                * telikelihood
                / 100.0
            )
        else:
            destroy_wt_epain = (
                0.5
                * (
                    (1.903 * 10**-13) * (teduration * 10**tepain_level)
                    + unbuilt_evalue / 4050000.0
                )
                * telikelihood
                / 100.0
            )
    else:
        destroy_wt_epain = 0.0
    destroy_wt = destroy_wt_pain + destroy_wt_epain
    return destroy_wt


def threat_longterm_health_fn():
    years = get_valid_float(
        "Expected duration of health issue (in years)? ", min_value=0
    )
    if years != 0:
        qol_change = get_valid_float(
            "Average quality of life decrease during health issue (0 to 2)? ",
            min_value=0,
            max_value=2,
        )
        income = get_valid_float(
            "Yearly income (in $) of person who would have health issue? ", min_value=0
        )
        age = get_valid_int(
            "Age (in years) of person who would have health issue? ",
            min_value=0,
            max_value=125,
        )
        potential_lost = get_valid_float(
            "Percent of full potential lost due to health issue? ",
            min_value=0,
            max_value=100,
        )
        tlikelihood = get_valid_float(
            "Perceived percent likelihood that person would make good on the threat to hurt? ",
            min_value=0,
            max_value=100,
        )
        work_years = 65 - age
        if work_years <= 0:
            destroy_wt = 0.5 * years * qol_change / 54.0 * tlikelihood / 100.0
        elif work_years >= years:
            destroy_wt = (
                0.5
                * (
                    years * income * (1.0 - potential_lost / 100.0) / 4050000.0
                    + years * qol_change / 54.0
                )
                * tlikelihood
                / 100.0
            )
        else:
            destroy_wt = (
                0.5
                * (
                    work_years * income * (1.0 - potential_lost / 100.0) / 4050000.0
                    + years * qol_change / 54.0
                )
                * tlikelihood
                / 100.0
            )
    else:
        destroy_wt = 0.0
    return destroy_wt


def threat_kill_fn():
    defaults = input(
        "Do you want to use default values for their life value (Y or N)? "
    )
    if defaults == "Y":
        lif_val = 1.54586
        life_expect = 78.27912
    else:
        age = get_valid_float(
            "Age (in years) of person who would die? ", min_value=0, max_value=125
        )
        qol = get_valid_float(
            "Average quality of remaining life (-1 to 1)? ", min_value=-1, max_value=1
        )
        income = get_valid_float("Yearly income (in $) of person who would die? ")
        n_interact = get_valid_float(
            "Average number of people per year they would have interacted with if they lived? ",
            min_value=0,
        )
        value_add = get_valid_float(
            "Average value add per person they would have interacted with (can be negative; enter 0.0001 if unsure)? "
        )
        emot_depth = get_valid_float(
            "Average depth of relationships, for the emotional loss felt (0 to 1)? ",
            min_value=0,
            max_value=1,
        )
        n_loss = get_valid_float(
            "How many relationships with people who would feel the emotional loss? ",
            min_value=0,
        )
        life_expect = (
            3.956 * 10**-5 * age**3 - 0.0023 * age**2 + 0.0767 * age + 76.98
        )
        if age < 66:
            lif_val = (
                (65 - age) * income / 4050000
                + (life_expect - age) * (qol / 54 + n_interact * value_add)
                + 0.01 * emot_depth * n_loss
            )
        elif age < 71:
            lif_val = (life_expect - age) * (
                qol / 54 + n_interact * value_add
            ) + 0.01 * emot_depth * n_loss
        else:
            lif_val = (life_expect - age) * (
                qol / 54 + n_interact * value_add
            ) + 0.001 * emot_depth * n_loss
    tlikelihood = get_valid_float(
        "Perceived percent likelihood that person would make good on the threat to kill? ",
        min_value=0,
        max_value=100,
    )
    destroy_wt = 0.5 * lif_val * tlikelihood / 100
    return destroy_wt


def restraint_fn():
    print(
        "Levels of restraint: 0 - none, 1 - exclude from somewhere via a wall, 2 - imprison, "
        "3 - move while unconscious, 4 - physically restrain, 5 - make unconscious"
    )
    restraint_lev = get_valid_int(
        "What level of restraint (0 through 5)? ", min_value=0, max_value=5
    )
    # Assume quality of life of 0.5
    if restraint_lev == 1:
        time_exclude = get_valid_float(
            "How much time excluded (in minutes)?", min_value=0
        )
        destroy_wt = 0.1 * time_exclude / 60 / 24 / 365 / 54 * 0.5
    elif restraint_lev == 2:
        time_imprison = get_valid_float(
            "How much time imprisoned (in minutes)?", min_value=0
        )
        destroy_wt = 0.5 * time_imprison / 60 / 24 / 365 / 54 * 0.5
    elif restraint_lev == 3:
        time_back = get_valid_float(
            "How much time to get back to original location (in minutes)? ", min_value=0
        )
        money_back = get_valid_float(
            "How much money to get back to original location (in $)? ", min_value=0
        )
        destroy_wt = 0.5 * time_back / 60 / 24 / 365 / 54 * 0.5 + money_back / 4050000
    elif restraint_lev == 4:
        time_restrain = get_valid_float(
            "How much time physically restrained (in minutes)?", min_value=0
        )
        destroy_wt = 1.0 * time_restrain / 60 / 24 / 365 / 54 * 0.5
    elif restraint_lev == 5:
        time_unconscious = get_valid_float(
            "How much time unconscious (in minutes)? ", min_value=0
        )
        destroy_wt = 1.0 * time_unconscious / 60 / 24 / 365 / 54 * 0.5
    else:
        destroy_wt = 0.0
    return destroy_wt

In [6]:
def calc_build(
    top_build,
    villain,
    villain_prop,
    percent_culp,
    harms,
    life_val,
    prop_val,
    harm_prop,
    num_pers,
    waiver,
):
    rights_upheld_wt = 0
    if top_build == 1:
        # 1 - Upholding rights
        print("For upholding rights:")
        top_build_wt1, top_build_wt2, top_build_wt3 = 0.0, 0.0, 0.0
        rtl_uphold = input("Is there an upholding of right to life (Y or N)? ")
        if rtl_uphold == "Y":
            viol_type = input(
                "Would this be preventing an actual violation (A) of right to life or just a threatened (T) one (A or T)? "
            )
            if viol_type != "A":
                tlikelihood = get_valid_float(
                    "Perceived percent likelihood that person would make good on the threat? ",
                    min_value=0,
                    max_value=100,
                )
            else:
                tlikelihood = 100.0
            if num_pers == 0:
                num_uphold = get_valid_int(
                    "How many people's right to life would've been violated at the same level of culpability? ",
                    min_value=0,
                )
                harms = get_valid_float(
                    "What percent of harm's way are they in? ",
                    min_value=0,
                    max_value=100,
                )
                if harms > 0:
                    percent_culp = get_valid_float(
                        "What's their percent culpability for getting themselves in harm's way? ",
                        min_value=0,
                        max_value=100,
                    )
                else:
                    percent_culp = 0.0
            else:
                num_uphold = num_pers
            worth_less = input(
                "Has the person made clear that they want their life to be worth less than those of others involved (Y or N)? "
            )
            if worth_less == "Y":
                want_valued = get_valid_float(
                    "What fraction do they want their right to life valued at with respect to others? (0 = not at all, 1 = equally) ",
                    min_value=0,
                    max_value=1,
                )
            else:
                want_valued = 1.0
            if villain == 1:
                # If they put someone's life or body integrity in harm's way, they're considered to have 1/10,000th of rights of someone 100% in harm's way
                harms = 100.0
                percent_culp = max(percent_culp, 99.99)
            elif villain_prop == 1:
                # If only put someone's property in harm's way, they're considered to have rights of someone 100% in harm's way
                harms = 100.0
                percent_culp = max(percent_culp, 0.0)
            # People 100% harm's way have right to life weight of 100, those 0% in harm's way have weight of 1,000,000
            top_build_wt1 = (
                num_uphold
                * want_valued
                * (tlikelihood / 100)
                * 1000000.0
                * (1 - (harms / 100) * (percent_culp / 100))
                / (1 + 9999 * (harms / 100))
            )
        rtbi_uphold = input(
            "Is there an upholding of right to body integrity (Y or N)? "
        )
        if rtbi_uphold == "Y":
            viol_type = input(
                "Would this be preventing an actual violation (A) of right to body integrity or just a threatened (T) one (A or T)? "
            )
            if viol_type != "A":
                tlikelihood = get_valid_float(
                    "Perceived percent likelihood that person would make good on the threat? ",
                    min_value=0,
                    max_value=100,
                )
            else:
                tlikelihood = 100.0
            if num_pers == 0:
                num_uphold = get_valid_int(
                    "How many people's right to body integrity would've been violated to the same level, including culpability? ",
                    min_value=0,
                )
                harms = get_valid_float(
                    "What percent of harm's way are they in? ",
                    min_value=0,
                    max_value=100,
                )
                if harms > 0:
                    percent_culp = get_valid_float(
                        "What's their percent culpability for getting themselves in harm's way? ",
                        min_value=0,
                        max_value=100,
                    )
                else:
                    percent_culp = 0.0
            else:
                num_uphold = num_pers
            print("For unwanted physical pain:")
            pain = avoid_pain_fn()
            print("For long-term health issues:")
            health = avoid_longterm_health_fn()
            print("For threat of physical violence or emotional pain:")
            tpain = threat_pain_fn()
            print(
                "For threat of physical violence to the point of a long-term health issue:"
            )
            thealth = threat_longterm_health_fn()
            print("For threat of physical violence to the point of death:")
            tkill = threat_kill_fn()
            print("For restraint:")
            restraint = restraint_fn()
            worth_less = input(
                "Has the person made clear that they want their body integrity to be worth less than those of others involved (Y or N)? "
            )
            if worth_less == "Y":
                want_valued = get_valid_float(
                    "How much do they want their right to body integrity valued with respect to others? (0 = not at all, 1 = equally) ",
                    min_value=0,
                    max_value=1,
                )
            else:
                want_valued = 1.0
            if villain == 1:
                # If they put someone's life or body integrity in harm's way, they're considered to have 1/10,000th of rights of someone 100% in harm's way
                harms = 100.0
                percent_culp = max(percent_culp, 99.99)
            elif villain_prop == 1:
                # If they only put someone's property in harm's way, they're considered to have rights of someone 100% in harm's way
                harms = 100.0
                percent_culp = max(percent_culp, 0.0)
            top_build_wt2 = (
                num_uphold
                * want_valued
                * (tlikelihood / 100)
                * (
                    1000000.0
                    * (1 - (harms / 100) * (percent_culp / 100))
                    * (pain + health + tpain + thealth + tkill + restraint)
                )
                / (1 + 9999 * (harms / 100))
            )
        rtp_uphold = input("Is there an upholding of right to property (Y or N)? ")
        if rtp_uphold == "Y":
            viol_type = input(
                "Would this be preventing an actual violation (A) of right to property or just a threatened (T) one (A or T)? "
            )
            if viol_type != "A":
                tlikelihood = get_valid_float(
                    "Perceived percent likelihood that person would make good on the threat? ",
                    min_value=0,
                    max_value=100,
                )
            else:
                tlikelihood = 100.0
            if num_pers == 0:
                num_uphold = get_valid_int(
                    "How many people's right to property would've been violated at the same level of culpability? ",
                    min_value=0,
                )
                harm_prop = get_valid_float(
                    "What percent of harm's way is their property in? ",
                    min_value=0,
                    max_value=100,
                )
            else:
                num_uphold = num_pers
            print("For property destruction and property stolen:")
            if harm_prop > 0:
                culp_prop = get_valid_float(
                    "What's the person's culpability for their property-risking situation, in percent? ",
                    min_value=0,
                    max_value=100,
                )
            else:
                culp_prop = 0.0
            if villain == 1:
                # If they put someone's life or body integrity in harm's way, their rights drop to 1/10,000th the value for property in harm's way
                harm_prop = 100.0
                culp_prop = max(culp_prop, 99.99)
            elif villain_prop == 1:
                # If they only put someone's property in harm's way, their rights drop to 1/5th the value for property in harm's way
                harm_prop = 100.0
                culp_prop = max(culp_prop, 80.0)
            top_build_wt3 = (
                num_uphold
                * (1 - harm_prop / 100 * culp_prop / 100)
                * (tlikelihood / 100)
                * (10000.0 * prop_val)
                / (1 + 99 * (harm_prop / 100))
            )
        top_build_wt = (top_build_wt1 + top_build_wt2 + top_build_wt3) * (
            1.0 - waiver / 100.0
        )
        rights_upheld_wt = top_build_wt
        print(f"*****Value build weight is: {top_build_wt}")
    elif top_build == 2:
        # 2 - Saving/maintaining a human life or reducing the probability of its loss
        print(
            "For saving/maintaining a human life or reducing the probability of its loss:"
        )
        if num_pers == 0:
            n_save = get_valid_int(
                "Statistical number of people who will live longer over the next 20 years? ",
                min_value=0,
            )
            waiver = get_valid_float(waiver_ask_build, min_value=0, max_value=100)
            build = delay_someone_dying_fn()
        else:
            n_save = num_pers
            build = life_val
        top_build_wt = n_save * build * (1.0 - waiver / 100.0)
        print(f"*****Value build weight is: {top_build_wt}")
    elif top_build == 3:
        # 3 - Saving a human from non-freely chosen physical pain, or reducing its probability
        print(
            "For saving a human from non-freely chosen physical pain, or reducing the probability of pain:"
        )
        if num_pers == 0:
            n_pain = get_valid_int(
                "Statistical number of people with less pain over the next 20 years? ",
                min_value=0,
            )
            waiver = get_valid_float(waiver_ask_build, min_value=0, max_value=100)
        else:
            n_pain = num_pers
        build = avoid_pain_fn()
        top_build_wt = n_pain * build * (1.0 - waiver / 100.0)
        print(f"*****Value build weight is: {top_build_wt}")
    elif top_build == 4:
        # 4 - Saving a human from long-term health issues, or reducing their probability
        print("For saving a human from long-term health issues:")
        if num_pers == 0:
            n_health = get_valid_int(
                "Statistical number of people with less long term health issues over the next 20 years? ",
                min_value=0,
            )
            waiver = get_valid_float(waiver_ask_build, min_value=0, max_value=100)
        else:
            n_health = num_pers
        build = avoid_longterm_health_fn()
        top_build_wt = n_health * build * (1.0 - waiver / 100.0)
        print(f"*****Value build weight is: {top_build_wt}")
    elif top_build == 5:
        # 5 - Repairing property
        print("For repairing property:")
        if num_pers == 0:
            n_prop_repair = get_valid_int(
                "How many effectively equivalent people would have their property damaged? ",
                min_value=0,
            )
            waiver = get_valid_float(waiver_ask_build, min_value=0, max_value=100)
        else:
            n_prop_repair = num_pers
        build = property_repair_fn()
        top_build_wt = n_prop_repair * build * (1.0 - waiver / 100.0)
        print(f"*****Value build weight is: {top_build_wt}")
    elif top_build == 6:
        # 6 - Going with one’s conscience
        print("For going with one’s conscience:")
        if num_pers == 0:
            n_consc = get_valid_int(
                "How many effectively equivalent people would go with their conscience? ",
                min_value=0,
            )
            waiver = get_valid_float(waiver_ask_build, min_value=0, max_value=100)
        else:
            n_consc = num_pers
        build = uphold_conscience_fn()
        top_build_wt = n_consc * build * (1.0 - waiver / 100.0)
        print(f"*****Value build weight is: {top_build_wt}")
    return top_build_wt, rights_upheld_wt

In [7]:
def delay_someone_dying_fn():
    # Decreased emotional loss, increased money making, increased human interaction due to delaying death
    defaults = input(
        "Do you want to use default values for their life value (Y or N)? "
    )
    if defaults == "Y":
        build_wt = 1.54586
    else:
        age = get_valid_float(
            "Average age (in years) of people who would die? ",
            min_value=0,
            max_value=125,
        )
        life_expect = get_valid_float(
            "New average life expectancy (in years) of people who would die (>age)? ",
            min_value=0,
            max_value=150,
        )
        qol = get_valid_float(
            "Average quality of life (-1 to 1) in their extra years? ",
            min_value=-1,
            max_value=1,
        )
        income = get_valid_float(
            "Average yearly income (in $) of people who would die? "
        )
        n_interact = get_valid_float(
            "Average number of people per year each person would interact with if they live? ",
            min_value=0,
        )
        value_add = get_valid_float(
            "Average value add per person they would interact with (enter 0.0001 if unsure)? "
        )
        n_loss = get_valid_float(
            "How many relationships each person had, on average, with people who would feel the emotional loss? ",
            min_value=0,
        )
        emot_depth = get_valid_float(
            "Average depth of relationships, for the emotional loss felt (0 to 1)? ",
            min_value=0,
            max_value=1,
        )
        if age < 66:
            build_wt = (
                (65 - age) * income / 4050000
                + (life_expect - age) * (qol / 54 + n_interact * value_add)
                + 0.001 * emot_depth * n_loss
            )
        elif age < 71:
            build_wt = (life_expect - age) * (
                qol / 54 + n_interact * value_add
            ) + 0.001 * emot_depth * n_loss
        else:
            build_wt = (life_expect - age) * (
                qol / 54 + n_interact * value_add
            ) + 0.0001 * emot_depth * n_loss
    return build_wt


def avoid_pain_fn():
    pain_level = get_valid_int(
        "Avoided physical pain level (1 to 10)? ", min_value=1, max_value=10
    )
    duration = get_valid_float(
        "Avoided duration of physical pain (in minutes)? ", min_value=0
    )
    built_value = get_valid_float(
        "Net value could build (in $) without the unwanted pain? ", min_value=0
    )
    if pain_level > 7:
        build_wt = (1.903 * 10**-6) * (
            duration * 100 ** (pain_level - 7)
        ) + built_value / 4050000.0
    else:
        build_wt = (1.903 * 10**-13) * (
            duration * 10**pain_level
        ) + built_value / 4050000.0
    return build_wt


def avoid_longterm_health_fn():
    years = get_valid_float(
        "Expected duration of health issue avoided (in years)? ",
        min_value=0,
        max_value=125,
    )
    qol_change = get_valid_float(
        "Average quality of life decrease would've had during health issue (0 to 2)? ",
        min_value=0,
        max_value=2,
    )
    income = get_valid_float(
        "Yearly income (in $) of person who would have health issue? ", min_value=0
    )
    age = get_valid_int(
        "Age (in years) of person who would have health issue? ",
        min_value=0,
        max_value=125,
    )
    potential_lost = get_valid_float(
        "Percent of full potential that would be lost due to health issue? ",
        min_value=0,
        max_value=100,
    )
    work_years = 65 - age
    if work_years <= 0:
        build_wt = years * qol_change / 54.0 * n_health
    elif work_years >= years:
        build_wt = (
            years * income * (1.0 - potential_lost / 100.0) / 4050000.0
            + years * qol_change / 54.0
        )
    else:
        build_wt = (
            work_years * income * (1.0 - potential_lost / 100.0) / 4050000.0
            + years * qol_change / 54.0
        )
    return build_wt


def property_repair_fn():
    cost = get_valid_float(
        "Increase in the replacement cost in dollars due to the property repair? ",
        min_value=0,
    )
    survive_change = get_valid_float(
        "Percent increases survival likelihood over the next year? ",
        min_value=0,
        max_value=100,
    )
    weight_death = get_valid_float(
        "Value weight of person(s) dying in the next year (enter 1.54586 if unsure)? "
    )
    egain = get_valid_float(
        "Weight of emotional gain if property had special meaning (enter 0.0001 if unsure)? "
    )
    morebuild = get_valid_float(
        "Likely increased future value building (in $) due to property regain, over next year? ",
        min_value=0,
    )
    build_wt = (
        cost / 4050000.0
        + survive_change / 100 * weight_death
        + egain
        + morebuild / 4050000.0
    )
    return build_wt


def uphold_conscience_fn():
    print(
        "Ethical breach levels: 1 - telling white lies, going against small values/ethics, "
        "2 - not helping others in significant need (who you don't see) when you could, "
        "3 - lying, stealing, cheating, not keeping your word, 4 - being significantly violent, "
        "not helping someone in significant need (right in front of you) when you could, "
        "5 - going against a large value/important ethic, such as not to murder, "
        "6 - extreme torturing"
    )
    breach_level = get_valid_int(
        "Level of ethical breach it would've been (1 to 6)? ", min_value=1, max_value=6
    )
    num_prev_breach = get_valid_int(
        "Previous number of breaches at this level? ", min_value=0
    )
    if breach_level == 1:
        build_wt = 0.0001 / (num_prev_breach + 1)
    elif breach_level == 2:
        build_wt = 0.005 / (num_prev_breach + 1)
    elif breach_level == 3:
        build_wt = 0.05 / (num_prev_breach + 1)
    elif breach_level == 4:
        build_wt = 0.5 / (num_prev_breach + 1)
    elif breach_level == 5:
        build_wt = 2.0 / (num_prev_breach + 1)
    else:
        build_wt = 5.0 / (num_prev_breach + 1)
    return build_wt

In [8]:
# These "demo" calculations are for relatively simple situations with only 1 harm type being
# considered at a time for % harm's way, and limited numbers of value destructions, builds,
# and decision options
option1 = input(
    "Please enter the first decision option (this should always be the option of doing nothing): "
)
option2 = input("Please enter the second decision option: ")
n_opt = 2
option3 = input(
    'Please enter the third decision option (or type "none" if there is no third option): '
)
if option3 != "none":
    n_opt = 3
    option4 = input(
        'Please enter the fourth decision option (or type "none" if there is no fourth option): '
    )
    if option4 != "none":
        n_opt = 4
prop_or_not = input("Do any of these options involve property loss (Y or N)? ")
option1_calc, option2_calc, option3_calc, option4_calc = 0.0, 0.0, 0.0, 0.0
rights_violations1, rights_violations2 = 0.0, 0.0
rights_violations3, rights_violations4 = 0.0, 0.0
person_number, life_value, prop_value = [], [], []
life_rights_opt2, life_rights_opt3, life_rights_opt4 = [], [], []
body_rights_opt2, body_rights_opt3, body_rights_opt4 = [], [], []
prop_rights_opt2, prop_rights_opt3, prop_rights_opt4 = [], [], []
prop_rights_fixed_opt2, prop_rights_fixed_opt3, prop_rights_fixed_opt4 = [], [], []
villain_to, villain_to_prop, villainy = [], [], []
percent_culpability, harms_way = [], []
villainy_prop, harms_way_prop, num_persons = [], [], []
pain_opt2, health_opt2, threat_pain_opt2 = [], [], []
threat_health_opt2, threat_kill_opt2 = [], []
restraint_opt2 = []
pain_opt3, health_opt3, threat_pain_opt3 = [], [], []
threat_health_opt3, threat_kill_opt3 = [], []
restraint_opt3 = []
pain_opt4, health_opt4, threat_pain_opt4 = [], [], []
threat_health_opt4, threat_kill_opt4 = [], []
restraint_opt4 = []
destroy_rights_wt2, destroy_rights_wt3, destroy_rights_wt4 = 0.0, 0.0, 0.0
n_people = get_valid_int(
    "How many different sets of people involved in the situation do you want to input info for? ",
    min_value=0,
)
for i in range(1, n_people + 1):
    person_number.append(i)
    print(f"For persons number {i}: ")
    num_pers = get_valid_int(
        "How many people are equivalent to this person, including them themselves? ",
        min_value=0,
    )
    num_persons.append(num_pers)
    defaults = input(
        "Do you want to use default values for this person's life value (Y or N)? "
    )
    if defaults == "Y":
        life_val = 1.54586
    else:
        life_val = life_value_fn()
    life_value.append(life_val)
    while True:
        villain_to_who = input(
            "Who did this person put in harm's way (enter the numbers of the people, separated by commas, or 0 for none)? "
        )
        if villain_to_who == "0":
            villainy.append(0)
            break
        try:
            person_list = [int(num.strip()) for num in villain_to_who.split(",")]
            villain_to.append(person_list)
            villainy.append(1)
        except ValueError:
            print(
                "Invalid input. Please enter valid integers separated by commas or '0' for none."
            )
            continue
    percent_culpability.append(
        get_valid_float(
            "What's their percent culpability for getting themselves in harm's way? ",
            min_value=0,
            max_value=100,
        )
    )
    harms_way.append(
        get_valid_float(
            "What percent of harm's way are they in? ", min_value=0, max_value=100
        )
    )
    if prop_or_not == "Y":
        while True:
            vill_to_prop = input(
                "Whose property did this person put in harm's way (enter the numbers of the people, separated by commas, or 0 for none)? "
            )
            if vill_to_prop == "0":
                villainy_prop.append(0)
                break
            try:
                prop_list = [int(num.strip()) for num in vill_to_prop.split(",")]
                villain_to_prop.append(prop_list)
                villainy_prop.append(1)
            except ValueError:
                print(
                    "Invalid input. Please enter valid integers separated by commas or '0' for none."
                )
                continue
        print(f"For the involved property that belongs to person {i}: ")
        prop_val = steal_and_prop_destroy_fn()
        prop_value.append(prop_val)
        harms_way_prop.append(
            get_valid_float(
                "What percent of harm's way is their property in? ",
                min_value=0,
                max_value=100,
            )
        )
    else:
        rtp_viol = "N"
        villain_to_prop.append([0])
        villainy_prop.append(0)
        prop_value.append(0.0)
        harms_way_prop.append(0.0)
for i in range(0, n_people):
    print(
        f"Please enter the right to life, right to body integrity, and/or right to property violations of Option 2 ({option2}) for person {i+1}."
    )
    rtl_viol = input("Is there a right to life violation (Y or N)? ")
    if rtl_viol == "Y":
        life_rights2 = life_rights_fn(
            villainy[i],
            villainy_prop[i],
            percent_culpability[i],
            harms_way[i],
            num_persons[i],
        )
    else:
        life_rights2 = 0.0
    rtbi_viol = input("Is there a right to body integrity violation (Y or N)? ")
    if rtbi_viol == "Y":
        (
            body_rights2,
            pain2,
            health2,
            threat_pain2,
            threat_health2,
            threat_kill2,
            restraint2,
        ) = body_rights_fn(
            villainy[i],
            villainy_prop[i],
            percent_culpability[i],
            harms_way[i],
            num_persons[i],
        )
    else:
        (
            body_rights2,
            pain2,
            health2,
            threat_pain2,
            threat_health2,
            threat_kill2,
            restraint2,
        ) = (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    if prop_or_not == "Y":
        rtp_viol = input("Is there a right to property violation (Y or N)? ")
    if rtp_viol == "Y":
        prop_rights2, prop_value2, prop_rights2_fixed = prop_rights_fn(
            villainy[i],
            villainy_prop[i],
            harms_way_prop[i],
            prop_value[i],
            num_persons[i],
        )
    else:
        prop_rights2, prop_value2, prop_rights2_fixed = 0.0, 0.0, 0.0
    life_rights_opt2.append(life_rights2)
    body_rights_opt2.append(body_rights2)
    pain_opt2.append(pain2)
    health_opt2.append(health2)
    threat_pain_opt2.append(threat_pain2)
    threat_health_opt2.append(threat_health2)
    threat_kill_opt2.append(threat_kill2)
    restraint_opt2.append(restraint2)
    prop_rights_opt2.append(prop_rights2)
    prop_rights_fixed_opt2.append(prop_rights2_fixed)
    destroy_rights_wt2 += num_persons[i] * (
        life_rights_opt2[i] + body_rights_opt2[i] + prop_rights_fixed_opt2[i]
    )
    if n_opt >= 3:
        print(
            f"Please enter the right to life, right to body integrity, and/or right to property violations of Option 3 ({option3}) for person {i+1}."
        )
        rtl_viol = input("Is there a right to life violation (Y or N)? ")
        if rtl_viol == "Y":
            life_rights3 = life_rights_fn(
                villainy[i],
                villainy_prop[i],
                percent_culpability[i],
                harms_way[i],
                num_persons[i],
            )
        else:
            life_rights3 = 0.0
        rtbi_viol = input("Is there a right to body integrity violation (Y or N)? ")
        if rtbi_viol == "Y":
            (
                body_rights3,
                pain3,
                health3,
                threat_pain3,
                threat_health3,
                threat_kill3,
                restraint3,
            ) = body_rights_fn(
                villainy[i],
                villainy_prop[i],
                percent_culpability[i],
                harms_way[i],
                num_persons[i],
            )
        else:
            (
                body_rights3,
                pain3,
                health3,
                threat_pain3,
                threat_health3,
                threat_kill3,
                restraint3,
            ) = (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
        if prop_or_not == "Y":
            rtp_viol = input("Is there a right to property violation (Y or N)? ")
        if rtp_viol == "Y":
            prop_rights3, prop_value3, prop_rights3_fixed = prop_rights_fn(
                villainy[i],
                villainy_prop[i],
                harms_way_prop[i],
                prop_value[i],
                num_persons[i],
            )
        else:
            prop_rights3, prop_value3, prop_rights3_fixed = 0.0, 0.0, 0.0
        life_rights_opt3.append(life_rights3)
        body_rights_opt3.append(body_rights3)
        pain_opt3.append(pain3)
        health_opt3.append(health3)
        threat_pain_opt3.append(threat_pain3)
        threat_health_opt3.append(threat_health3)
        threat_kill_opt3.append(threat_kill3)
        restraint_opt3.append(restraint3)
        prop_rights_opt3.append(prop_rights3)
        prop_rights_fixed_opt3.append(prop_rights3_fixed)
        destroy_rights_wt3 += num_persons[i] * (
            life_rights_opt3[i] + body_rights_opt3[i] + prop_rights_fixed_opt3[i]
        )
    if n_opt == 4:
        print(
            f"Please enter the right to life, right to body integrity, and/or right to property violations of Option 4 ({option4}) for person {i+1}."
        )
        rtl_viol = input("Is there a right to life violation (Y or N)? ")
        if rtl_viol == "Y":
            life_rights4 = life_rights_fn(
                villainy[i],
                villainy_prop[i],
                percent_culpability[i],
                harms_way[i],
                num_persons[i],
            )
        else:
            life_rights4 = 0.0
        rtbi_viol = input("Is there a right to body integrity violation (Y or N)? ")
        if rtbi_viol == "Y":
            (
                body_rights4,
                pain4,
                health4,
                threat_pain4,
                threat_health4,
                threat_kill4,
                restraint4,
            ) = body_rights_fn(
                villainy[i],
                villainy_prop[i],
                percent_culpability[i],
                harms_way[i],
                num_persons[i],
            )
        else:
            (
                body_rights4,
                pain4,
                health4,
                threat_pain4,
                threat_health4,
                threat_kill4,
                restraint4,
            ) = (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
        if prop_or_not == "Y":
            rtp_viol = input("Is there a right to property violation (Y or N)? ")
        if rtp_viol == "Y":
            prop_rights4, prop_value4, prop_rights4_fixed = prop_rights_fn(
                villainy[i],
                villainy_prop[i],
                harms_way_prop[i],
                prop_value[i],
                num_persons[i],
            )
        else:
            prop_rights4, prop_value4, prop_rights4_fixed = 0.0, 0.0, 0.0
        life_rights_opt4.append(life_rights4)
        body_rights_opt4.append(body_rights4)
        pain_opt4.append(pain4)
        health_opt4.append(health4)
        threat_pain_opt4.append(threat_pain4)
        threat_health_opt4.append(threat_health4)
        threat_kill_opt4.append(threat_kill4)
        restraint_opt4.append(restraint4)
        prop_rights_opt4.append(prop_rights4)
        prop_rights_fixed_opt4.append(prop_rights4_fixed)
        destroy_rights_wt4 += num_persons[i] * (
            life_rights_opt4[i] + body_rights_opt4[i] + prop_rights_fixed_opt4[i]
        )
for ii in range(2, n_opt + 1):
    # Ask user to input any remaining, relevant value destructions after printing a list of them
    print("")
    if ii == 2:
        numth = "second"
        opt = option2
    if ii == 3:
        numth = "third"
        opt = option3
    if ii == 4:
        numth = "fourth"
        opt = option4
    print(f"For the {numth} decision option ({opt}):")
    print("Please enter up to 12 value destructions from the list below:")
    file_path = "/Files/ValueDestructionsListShort.txt"
    try:
        with open(file_path, "r") as file:
            destroy_dict = {}
            for line in file:
                parts = line.strip().split("-")
                if len(parts) == 2:
                    number = int(parts[0].strip())
                    option = parts[1].strip()
                    destroy_dict[number] = option
            for number, option in destroy_dict.items():
                print(f"{number} - {option}")
    except FileNotFoundError:
        print(f'File "{file_path}" not found.')
    except Exception as e:
        print(f"An error occurred while trying to print the list: {str(e)}")
    # Check input for integers being in the right range:
    inputs = []  # List to store the collected inputs
    waivers = []
    person_destroy = []
    for i in range(12):
        while True:
            try:
                user_input = int(
                    input(
                        f"Value destruction #{i + 1} (integer from 1 to 5, or 0 if finished)? "
                    )
                )
                if user_input == 0:
                    break
                elif 1 <= user_input <= 5:
                    inputs.append(user_input)
                    person_destroy.append(
                        int(
                            input(
                                "Which person number does this value destruction correspond to (enter 0 if none)? "
                            )
                        )
                    )
                    break  # Exit the inner loop if a valid input is provided
                else:
                    print("Invalid input. Please enter an integer from 0 to 5.")
            except ValueError:
                print("Invalid input. Please enter a valid integer.")
        if user_input == 0:
            break  # Exit the outer loop if the user enters zero
        waiv = get_valid_float(waiver_ask_destroy, min_value=0, max_value=100)
        waivers.append(waiv)
    input_wts = (
        []
    )  # List to store the value weights corresponding to the input categories
    top_destroy_tot_wt = 0.0
    rights_violations_tot_wt = 0.0
    for i in range(len(inputs)):
        print("")
        if person_destroy[i] != 0:
            top_destroy_wt = calc_destroy(
                inputs[i],
                num_persons[person_destroy[i] - 1],
                life_value[person_destroy[i] - 1],
                waivers[i],
                prop_value[person_destroy[i] - 1],
            )
        else:
            top_destroy_wt = calc_destroy(inputs[i], 0, 0, 0, 0)
        input_wts.append(top_destroy_wt)
        fix_intent_or_not = input(
            "Is there intent to repair this value destruction later (Y/N)? "
        )
        if fix_intent_or_not == "Y":
            likelihood_fix = get_valid_float(
                "What is the percent likelihood of this value destruction repair occurring? ",
                min_value=0,
                max_value=100,
            )
            percent_fix = get_valid_float(
                "What percentage of the value destruction is likely to be repaired? ",
                min_value=0,
                max_value=100,
            )
        else:
            likelihood_fix = 0
            percent_fix = 0
        top_destroy_tot_wt += input_wts[i] * (
            1 - (likelihood_fix / 100 * percent_fix / 100)
        )

    # Ask user to input the relevant value builds after printing a list of them
    print("")
    print("Please enter up to 12 value builds from the list below:")
    file_path2 = "/Files/ValueBuildsListShort.txt"
    try:
        with open(file_path2, "r") as file2:
            build_dict = {}
            for line2 in file2:
                parts = line2.strip().split("-")
                if len(parts) == 2:
                    number = int(parts[0].strip())
                    option = parts[1].strip()
                    build_dict[number] = option
            for number, option in build_dict.items():
                print(f"{number} - {option}")
    except FileNotFoundError:
        print(f"File '{file_path2}' not found.")
    except Exception as e:
        print(f"An error occurred while trying to print the list: {str(e)}")
    # Check input for integers being in the right range:
    inputs2 = []  # List to store the collected inputs
    waivers2 = []
    person_build = []
    for i in range(12):
        while True:
            try:
                user_input = int(
                    input(
                        f"Value build #{i + 1} (integer from 1 to 6, or 0 if finished)? "
                    )
                )
                if user_input == 0:
                    break
                elif 1 <= user_input <= 6:
                    inputs2.append(user_input)
                    person_build.append(
                        int(
                            input(
                                "Which person number does this value build correspond to (enter 0 if none)? "
                            )
                        )
                    )
                    break  # Exit the inner loop if a valid input is provided
                else:
                    print("Invalid input. Please enter an integer from 0 to 6.")
            except ValueError:
                print("Invalid input. Please enter a valid integer.")
        if user_input == 0:
            break  # Exit the outer loop if the user enters zero
        waiv = get_valid_float(waiver_ask_build, min_value=0, max_value=100)
        waivers2.append(waiv)
    input2_wts = (
        []
    )  # List to store the value weights corresponding to the input categories
    top_build_tot_wt = 0.0
    for i in range(len(inputs2)):
        print("")
        if person_build[i] != 0:
            jj = person_build[i] - 1
            top_build_wt, rights_upheld_wt = calc_build(
                inputs2[i],
                villainy[jj],
                villainy_prop[jj],
                percent_culpability[jj],
                harms_way[jj],
                life_value[jj],
                prop_value[jj],
                harms_way_prop[jj],
                num_persons[jj],
                waivers2[i],
            )
        else:
            top_build_wt, rights_upheld_wt = calc_build(
                inputs2[i],
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0,
                0.0,
            )
        input2_wts.append(top_build_wt)
        destroy_intent_or_not = input(
            "Is there intent to destroy this value build later (Y/N)? "
        )
        if destroy_intent_or_not == "Y":
            likelihood_destroy = get_valid_float(
                "What is the percent likelihood of this value build destruction occurring? ",
                min_value=0,
                max_value=100,
            )
            percent_destroy = get_valid_float(
                "What percentage of the value build is likely to be destroyed? ",
                min_value=0,
                max_value=100,
            )
        else:
            likelihood_destroy = 0.0
            percent_destroy = 0.0
        top_build_tot_wt += input2_wts[i] * (
            1 - (likelihood_destroy / 100 * percent_destroy / 100)
        )
        rights_violations_tot_wt -= rights_upheld_wt

    # Calculate net value builds/destroys
    if ii == 2:
        option2_calc = -top_destroy_tot_wt + top_build_tot_wt - destroy_rights_wt2
        rights_violations2 = rights_violations_tot_wt + destroy_rights_wt2
        top_destroy_tot_wt2 = top_destroy_tot_wt
        top_build_tot_wt2 = top_build_tot_wt
    elif ii == 3:
        option3_calc = -top_destroy_tot_wt + top_build_tot_wt - destroy_rights_wt3
        rights_violations3 = rights_violations_tot_wt + destroy_rights_wt3
        top_destroy_tot_wt3 = top_destroy_tot_wt
        top_build_tot_wt3 = top_build_tot_wt
    elif ii == 4:
        option4_calc = -top_destroy_tot_wt + top_build_tot_wt - destroy_rights_wt4
        rights_violations4 = rights_violations_tot_wt + destroy_rights_wt4
        top_destroy_tot_wt4 = top_destroy_tot_wt
        top_build_tot_wt4 = top_build_tot_wt

print("")
# Identify and output the best option in terms of most value build/least value destruction
# Also for least rights violations
if n_opt == 2:
    maximum = max(option1_calc, option2_calc)
    min_rights_viol = min(rights_violations1, rights_violations2)
    if maximum == option1_calc:
        if option1_calc == option2_calc:
            print(
                f"*****The ethical best options are calculated to be: {option1} and {option2}"
            )
        else:
            print(f"*****The ethical best option is calculated to be: {option1}")
    else:
        print(f"*****The ethical best option is calculated to be: {option2}")
    if min_rights_viol == rights_violations1:
        if rights_violations1 == rights_violations2:
            print(
                f"*****The least overall rights-violating options are calculated to be: {option1} and {option2}"
            )
        else:
            print(
                f"*****The least overall rights-violating option is calculated to be: {option1}"
            )
    else:
        print(
            f"*****The least overall rights-violating option is calculated to be: {option2}"
        )
elif n_opt == 3:
    maximum = max(option1_calc, option2_calc, option3_calc)
    min_rights_viol = min(rights_violations1, rights_violations2, rights_violations3)
    if option1_calc == option2_calc and option1_calc == option3_calc:
        print(
            f"*****The ethical best options are calculated to be: all {n_opt} options"
        )
    elif maximum == option1_calc:
        if option1_calc == option2_calc:
            print(
                f"*****The ethical best options are calculated to be: {option1} and {option2}"
            )
        elif option1_calc == option3_calc:
            print(
                f"*****The ethical best options are calculated to be: {option1} and {option3}"
            )
        else:
            print(f"*****The ethical best option is calculated to be: {option1}")
    elif maximum == option2_calc:
        if option2_calc == option3_calc:
            print(
                f"*****The ethical best options are calculated to be: {option2} and {option3}"
            )
        else:
            print(f"*****The ethical best option is calculated to be: {option2}")
    else:
        print(f"*****The ethical best option is calculated to be: {option3}")
    if (
        rights_violations1 == rights_violations2
        and rights_violations1 == rights_violations3
    ):
        print(
            f"*****The least overall rights-violating options are calculated to be: all {n_opt} options"
        )
    elif min_rights_viol == rights_violations1:
        if rights_violations1 == rights_violations2:
            print(
                f"*****The least overall rights-violating options are calculated to be: {option1} and {option2}"
            )
        elif rights_violations1 == rights_violations3:
            print(
                f"*****The least overall rights-violating options are calculated to be: {option1} and {option3}"
            )
        else:
            print(
                f"*****The least overall rights-violating option is calculated to be: {option1}"
            )
    elif min_rights_viol == rights_violations2:
        if rights_violations2 == rights_violations3:
            print(
                f"*****The least overall rights-violating options are calculated to be: {option2} and {option3}"
            )
        else:
            print(
                f"*****The least overall rights-violating option is calculated to be: {option2}"
            )
    else:
        print(
            f"*****The least overall rights-violating option is calculated to be: {option3}"
        )
elif n_opt == 4:
    max_opt_all = []
    max_val = max(option1_calc, option2_calc, option3_calc, option4_calc)
    if max_val == option1_calc:
        max_opt_all.append(option1)
        if option1_calc == option2_calc:
            max_opt_all.append(option2)
        if option1_calc == option3_calc:
            max_opt_all.append(option3)
        if option1_calc == option4_calc:
            max_opt_all.append(option4)
    elif max_val == option2_calc:
        max_opt_all.append(option2)
        if option2_calc == option3_calc:
            max_opt_all.append(option3)
        if option2_calc == option4_calc:
            max_opt_all.append(option4)
    elif max_val == option3_calc:
        max_opt_all.append(option3)
        if option3_calc == option4_calc:
            max_opt_all.append(option4)
    else:
        max_opt_all.append(option4)
    if len(max_opt_all) == 1:
        print(f"*****The ethical best option is calculated to be: {max_opt_all[0]}")
    elif len(max_opt_all) == 2:
        print(
            f"*****The ethical best options are calculated to be: {max_opt_all[0]} and "
            f"{max_opt_all[1]}"
        )
    elif len(max_opt_all) == 3:
        print(
            f"*****The ethical best options are calculated to be: {max_opt_all[0]}, "
            f"{max_opt_all[1]} and {max_opt_all[2]}"
        )
    else:
        print(
            f"*****The ethical best options are calculated to be: all {n_opt} options"
        )
    min_rights_all = []
    min_rights_viol = min(
        rights_violations1, rights_violations2, rights_violations3, rights_violations4
    )
    if min_rights_viol == rights_violations1:
        min_rights_all.append(option1)
        if rights_violations1 == rights_violations2:
            min_rights_all.append(option2)
        if rights_violations1 == rights_violations3:
            min_rights_all.append(option3)
        if rights_violations1 == rights_violations4:
            min_rights_all.append(option4)
    elif min_rights_viol == rights_violations2:
        min_rights_all.append(option2)
        if rights_violations2 == rights_violations3:
            min_rights_all.append(option3)
        if rights_violations2 == rights_violations4:
            min_rights_all.append(option4)
    elif min_rights_viol == rights_violations3:
        min_rights_all.append(option3)
        if rights_violations3 == rights_violations4:
            min_rights_all.append(option4)
    else:
        min_rights_all.append(option4)
    if len(min_rights_all) == 1:
        print(
            f"*****The least overall rights-violating option is calculated to be: {min_rights_all[0]}"
        )
    elif len(min_rights_all) == 2:
        print(
            f"*****The least overall rights-violating options are calculated to be: {min_rights_all[0]} and "
            f"{min_rights_all[1]}"
        )
    elif len(min_rights_all) == 3:
        print(
            f"*****The least overall rights-violating options are calculated to be: {min_rights_all[0]}, "
            f"{min_rights_all[1]} and {min_rights_all[2]}"
        )
    else:
        print(
            f"*****The least overall rights-violating options are calculated to be: all {n_opt} options"
        )

print(
    "*****The estimated net value builds (positive) or destructions (negative) for each option are:"
)
print(f"{option1_calc} for {option1}")
print(
    f"{option2_calc} for {option2} (destroy = -{top_destroy_tot_wt2 + destroy_rights_wt2}, build = {top_build_tot_wt2})"
)
if n_opt >= 3:
    print(
        f"{option3_calc} for {option3} (destroy = -{top_destroy_tot_wt3 + destroy_rights_wt3}, build = {top_build_tot_wt3})"
    )
if n_opt == 4:
    print(
        f"{option4_calc} for {option4} (destroy = -{top_destroy_tot_wt4 + destroy_rights_wt4}, build = {top_build_tot_wt4})"
    )

Please enter the first decision option (this should always be the option of doing nothing): Nothing
Please enter the second decision option: Turn
Please enter the third decision option (or type "none" if there is no third option): none
Do any of these options involve property loss (Y or N)? N
How many different sets of people involved in the situation do you want to input info for? 2
For persons number 1: 
How many people are equivalent to this person, including them themselves? 5
Do you want to use default values for this person's life value (Y or N)? Y
Who did this person put in harm's way (enter the numbers of the people, separated by commas, or 0 for none)? 0
What's their percent culpability for getting themselves in harm's way? 50
What percent of harm's way are they in? 100
For persons number 2: 
How many people are equivalent to this person, including them themselves? 1
Do you want to use default values for this person's life value (Y or N)? Y
Who did this person put in harm's wa